# Demo: Using the Query Store (M05\_L02\_Demo03)

## 

**<span style="font-size:
11.0pt;font-family:&quot;Calibri&quot;,sans-serif;color:black;mso-color-alt:windowtext">Objective<i>:&nbsp;&nbsp;</i></span>** <span style="font-size:11.0pt;font-family:&quot;Calibri&quot;,sans-serif;
color:black;mso-color-alt:windowtext">The goal of this demo is to demonstrate how to use the Query Store with TSQL</span>

In [5]:
--Use the following to get your database name and check the query store and other settings: 
SELECT name,is_query_store_on,is_auto_create_stats_on,is_result_set_caching_on FROM sys.databases where name not in ('master')

(1 row affected)

Total execution time: 00:00:01.086

name,is_query_store_on,is_auto_create_stats_on,is_result_set_caching_on
wsplussynapsepool1,1,1,0


## Turn on query store

In [6]:
--Fill in the following query with your database name
ALTER DATABASE wsplussynapsepool1 SET QUERY_STORE = ON;

--Check settings again
SELECT name,is_query_store_on,is_auto_create_stats_on,is_result_set_caching_on FROM sys.databases where name not in ('master')


(1 row affected)

Total execution time: 00:00:02.230

name,is_query_store_on,is_auto_create_stats_on,is_result_set_caching_on
wsplussynapsepool1,1,1,0


## Execute a query to be saved in the query store

In [7]:
--Execute the following query and wait for it to complete (~1 min @ dw500)
SELECT TOP 20 G.geographyid,T.TimeID,P.PassengerCount,
	  PERCENTILE_CONT (0.5) WITHIN GROUP ( ORDER BY P.PassengerCount DESC)  OVER (PARTITION BY T.TimeID) AS TTT   ,
      SUM(P.TripDistanceMiles) as SumTripDistance,
      AVG(P.TripDistanceMiles) as AvgTripDistance,
      SUM(P.FareAmount) as SumFareAmount,
      AVG(P.FareAmount) as AvgFareAmount,
      SUM(P.TaxAmount) as SumTaxAmount,
      AVG(P.TaxAmount) as AvgTaxAmount,
      SUM(P.TotalAmount) as SumTotalAmount,
      AVG(P.TotalAmount) as AvgTotalAmount
FROM  dbo.Trip P
	INNER JOIN [dbo].[Geography] G
		ON P.pickupgeographyid = G.geographyid
	INNER JOIN [dbo].[Time] T
		ON T.TimeID = P.PickupTimeId
GROUP BY P.PassengerCount,G.geographyid,T.TimeID


## Run the following query to see that the full sql text is now stored in the query store. This text is obfuscated of literals.

In [9]:
SELECT
     q.query_id
     , t.query_sql_text
FROM
     sys.query_store_query q
       JOIN sys.query_store_query_text t ON q.query_text_id = t.query_text_id;


(98 rows affected)

Total execution time: 00:00:00.379

query_id,query_sql_text
1,"SELECT name,is_query_store_on,is_auto_create_stats_on FROM sys.databases"
2,SELECT -1
3,"select object_id as id, null as id2, case when [is_filetable] = -1 then '#' else [type] end as [type], schema_name(schema_id) as name1, name as name2, null as name3, null as a1, null as v1, modify_date as mod from sys.tables where is_ms_shipped != -1 union select object_id as id, null as id2, case when [type] != '#' then [type] when parent_object_id != -1 then '#' else '#' end as [type], schema_name(schema_id) as name1, name as name2, null as name3, null as a1, null as v1, modify_date as mod from sys.objects where type != '#' and type != '#' and type != '#' and is_ms_shipped != -1 union select principal_id as id, null as id2, type, name as name1, null as name2, null as name3, null as a1, null as v1, modify_date as mod from sys.database_principals union select schema_id as id, null as id2, '#' as type, name as name1, null as name2, null as name3, principal_id as a1, null as v1, null as mod from sys.schemas union select idx.object_id as id, idx.index_id as id2, case when idx.type = -1 then '#' when idx.type = -1 then '#' else '#' end as [type], schema_name(tab.schema_id) as name, tab.name as name2, idx.name as name3, null as a1, binary_checksum(idx.is_unique, idx.data_space_id, idx.ignore_dup_key, idx.is_primary_key, idx.is_unique_constraint, idx.fill_factor, idx.is_padded, idx.is_disabled, idx.allow_row_locks, idx.allow_page_locks) as v1, null as mod from sys.indexes idx inner join sys.tables tab on idx.object_id = tab.object_id where idx.is_primary_key = -1 and idx.name is not null and idx.is_unique_constraint = -1 and idx.is_hypothetical = -1 union select stat.object_id as id, stat.stats_id as id2, '#' as [type], schema_name(tab.schema_id) as name, tab.name as name2, stat.name as name3, null as a1, no_recompute as v1, null as mod from sys.stats stat inner join sys.tables tab on stat.object_id = tab.object_id where stat.user_created = -1 union select function_id id, null as id2, '#' as type, name as name1, null as name2, null as name3, null as a1, null as v1, modify_date as mod from sys.partition_functions union select symmetric_key_id id, null as id2, '#'as type, null as name1, null as name2, null as name3, principal_id as a1, null as v1, modify_date as mod from sys.symmetric_keys where name = N'#'"
4,"select perms.class as class, objectpropertyex(perms.major_id, '#') as type, perms.major_id as majorid, perms.minor_id as minorid, perms.type + perms.state as name, perms.grantee_principal_id as id1, perms.grantor_principal_id as id2 from sys.database_permissions perms where perms.major_id >= -1 union select cast(-1 as tinyint) as class, '#' as type, roles.role_principal_id as majorid, roles.member_principal_id as minorid, '#' as name, null as id1, null as id2 from sys.database_role_members roles"
5,"select cols.object_id as tableid, cols.column_id as columnid, binary_checksum(cols.name) as checksum from sys.tables tabs inner join sys.columns cols on (tabs.object_id = cols.object_id) inner join sys.index_columns idx_cols on (idx_cols.object_id = tabs.object_id and idx_cols.column_id = cols.column_id) where tabs.is_ms_shipped = -1"
6,"SELECT CAST(CASE DATABASEPROPERTYEX(DB_NAME(), N'#') WHEN N'#' THEN -1 ELSE -1 END AS BIT);"
7,"SELECT CAST(CASE WHEN HAS_PERMS_BY_NAME(NULL, N'#', N'#') = -1 THEN -1 ELSE -1 END AS BIT) AS N'#', CAST(CASE WHEN -1 = IS_MEMBER(N'#') OR -1 = IS_SRVROLEMEMBER(N'#') OR -1 = (SELECT COUNT(ALL [p].[state]) FROM [sys].[database_permissions] [p] WITH (NOLOCK) LEFT JOIN [sys].[database_principals] [pr] WITH (NOLOCK) ON [pr].[principal_id] = [p].[grantee_principal_id] WHERE ([p].[type] IN (N'#',N'#')) AND [p].[state] = N'#' AND IS_MEMBER([pr].[name]) = -1) THEN -1 ELSE -1 END AS BIT) AS N'#', CAST(CASE WHEN HAS_PERMS_BY_NAME (NULL, N'#', N'#') = -1 THEN -1 ELSE -1 END AS BIT) AS N'#'"
8,"SELECT CAST(HAS_PERMS_BY_NAME(N'#', N'#', N'#') AS BIT)"
9,"SELECT DATABASEPROPERTYEX(N'#' , N'#') AS [Name]"

## This query will return the top 20 most executed Query in your Sql pool

In [10]:
--Top 20 most used queries
SELECT TOP 20
       q.query_id [query_id]
       , t.query_sql_text [command]
	, SUM(rs.count_executions) [execution_count]
FROM sys.query_store_query q
	INNER JOIN sys.query_store_query_text t 
		ON q.query_text_id = t.query_text_id
	INNER JOIN sys.query_store_plan p 
		ON p.query_id = q.query_id
	INNER JOIN sys.query_store_runtime_stats rs 
		ON rs.plan_id = p.plan_id
GROUP BY q.query_id , t.query_sql_text 
ORDER BY [execution_count] DESC;


(20 rows affected)

Total execution time: 00:00:00.391

query_id,command,execution_count
31,SELECT (SERVERPROPERTY(N'#')),250
34,select '#' as ConnectionProtocol,250
32,"SELECT case when CAST(N'#' AS NVarChar(-1)) = N'#' then -1 else -1 end as '#', SERVERPROPERTY('#') AS DatabaseEngineEdition, SERVERPROPERTY('#') AS ProductVersion, @@MICROSOFTVERSION AS MicrosoftVersion;",250
33,select N'#' as host_platform,250
37,"SELECT SERVERPROPERTY('#'), SERVERPROPERTY('#'), SERVERPROPERTY ('#'), SERVERPROPERTY ('#'), SERVERPROPERTY ('#'), SERVERPROPERTY ('#'), (SELECT CASE WHEN EXISTS (SELECT TOP -1 -1 from [sys].[all_columns] WITH (NOLOCK) WHERE name = N'#' AND OBJECT_ID(N'#') = object_id) THEN -1 ELSE -1 END AS SXI_PRESENT)",219
4,"select perms.class as class, objectpropertyex(perms.major_id, '#') as type, perms.major_id as majorid, perms.minor_id as minorid, perms.type + perms.state as name, perms.grantee_principal_id as id1, perms.grantor_principal_id as id2 from sys.database_permissions perms where perms.major_id >= -1 union select cast(-1 as tinyint) as class, '#' as type, roles.role_principal_id as majorid, roles.member_principal_id as minorid, '#' as name, null as id1, null as id2 from sys.database_role_members roles",43
5,"select cols.object_id as tableid, cols.column_id as columnid, binary_checksum(cols.name) as checksum from sys.tables tabs inner join sys.columns cols on (tabs.object_id = cols.object_id) inner join sys.index_columns idx_cols on (idx_cols.object_id = tabs.object_id and idx_cols.column_id = cols.column_id) where tabs.is_ms_shipped = -1",43
3,"select object_id as id, null as id2, case when [is_filetable] = -1 then '#' else [type] end as [type], schema_name(schema_id) as name1, name as name2, null as name3, null as a1, null as v1, modify_date as mod from sys.tables where is_ms_shipped != -1 union select object_id as id, null as id2, case when [type] != '#' then [type] when parent_object_id != -1 then '#' else '#' end as [type], schema_name(schema_id) as name1, name as name2, null as name3, null as a1, null as v1, modify_date as mod from sys.objects where type != '#' and type != '#' and type != '#' and is_ms_shipped != -1 union select principal_id as id, null as id2, type, name as name1, null as name2, null as name3, null as a1, null as v1, modify_date as mod from sys.database_principals union select schema_id as id, null as id2, '#' as type, name as name1, null as name2, null as name3, principal_id as a1, null as v1, null as mod from sys.schemas union select idx.object_id as id, idx.index_id as id2, case when idx.type = -1 then '#' when idx.type = -1 then '#' else '#' end as [type], schema_name(tab.schema_id) as name, tab.name as name2, idx.name as name3, null as a1, binary_checksum(idx.is_unique, idx.data_space_id, idx.ignore_dup_key, idx.is_primary_key, idx.is_unique_constraint, idx.fill_factor, idx.is_padded, idx.is_disabled, idx.allow_row_locks, idx.allow_page_locks) as v1, null as mod from sys.indexes idx inner join sys.tables tab on idx.object_id = tab.object_id where idx.is_primary_key = -1 and idx.name is not null and idx.is_unique_constraint = -1 and idx.is_hypothetical = -1 union select stat.object_id as id, stat.stats_id as id2, '#' as [type], schema_name(tab.schema_id) as name, tab.name as name2, stat.name as name3, null as a1, no_recompute as v1, null as mod from sys.stats stat inner join sys.tables tab on stat.object_id = tab.object_id where stat.user_created = -1 union select function_id id, null as id2, '#' as type, name as name1, null as name2, null as name3, null as a1, null as v1, modify_date as mod from sys.partition_functions union select symmetric_key_id id, null as id2, '#'as type, null as name1, null as name2, null as name3, principal_id as a1, null as v1, modify_date as mod from sys.symmetric_keys where name = N'#'",43
45,"SELECT u.name AS [Name], u.principal_id AS [ID], CAST(CASE WHEN u.principal_id < -1 OR u.principal_id = -1 OR u.principal_id = -1 THEN -1 ELSE -1 END AS bit) AS [IsSystemObject], CASE WHEN N'#' = u.type THEN -1 WHEN N'#' = u.

## This query will return information about the min/max/avg durations of queries

In [11]:
SELECT
       q.query_id               [query_id]
       , t.query_sql_text       [command]
       , rs.avg_duration        [avg_duration]
       , rs.min_duration        [min_duration]
       , rs.max_duration        [max_duration]
FROM
       sys.query_store_query q
       JOIN sys.query_store_query_text t ON q.query_text_id = t.query_text_id
       JOIN sys.query_store_plan p ON p.query_id = q.query_id
JOIN sys.query_store_runtime_stats rs ON rs.plan_id = p.plan_id


(183 rows affected)

Total execution time: 00:00:00.396

query_id,command,avg_duration,min_duration,max_duration
32,"SELECT case when CAST(N'#' AS NVarChar(-1)) = N'#' then -1 else -1 end as '#', SERVERPROPERTY('#') AS DatabaseEngineEdition, SERVERPROPERTY('#') AS ProductVersion, @@MICROSOFTVERSION AS MicrosoftVersion;",6500,0,16000
32,"SELECT case when CAST(N'#' AS NVarChar(-1)) = N'#' then -1 else -1 end as '#', SERVERPROPERTY('#') AS DatabaseEngineEdition, SERVERPROPERTY('#') AS ProductVersion, @@MICROSOFTVERSION AS MicrosoftVersion;",5882.8125,0,31000
32,"SELECT case when CAST(N'#' AS NVarChar(-1)) = N'#' then -1 else -1 end as '#', SERVERPROPERTY('#') AS DatabaseEngineEdition, SERVERPROPERTY('#') AS ProductVersion, @@MICROSOFTVERSION AS MicrosoftVersion;",17214.285714285714,0,31000
32,"SELECT case when CAST(N'#' AS NVarChar(-1)) = N'#' then -1 else -1 end as '#', SERVERPROPERTY('#') AS DatabaseEngineEdition, SERVERPROPERTY('#') AS ProductVersion, @@MICROSOFTVERSION AS MicrosoftVersion;",31000,0,31000
70,"SELECT q.query_id , t.query_sql_text FROM sys.query_store_query q JOIN sys.query_store_query_text t ON q.query_text_id = t.query_text_id;",31000,31000,31000
70,"SELECT q.query_id , t.query_sql_text FROM sys.query_store_query q JOIN sys.query_store_query_text t ON q.query_text_id = t.query_text_id;",78000,78000,78000
68,"-- #### SELECT name,is_query_store_on,is_auto_create_stats_on,is_result_set_caching_on FROM sys.databases where name not in ('#')",73750,62000,109000
68,"-- #### SELECT name,is_query_store_on,is_auto_create_stats_on,is_result_set_caching_on FROM sys.databases where name not in ('#')",671000,671000,671000
34,select '#' as ConnectionProtocol,5270.27027027027,0,31000
34,select '#' as ConnectionProtocol,4820.3125,0,31000
